<a href="https://colab.research.google.com/github/AkshatThecoder/Emotion-Detection-from-Text/blob/main/Emotion_Detection_from_Text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
#DATASET USED: https://www.kaggle.com/datasets/pashupatigupta/emotion-detection-from-text

!pip install neattext scikit-learn imbalanced-learn joblib --quiet

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import neattext.functions as nfx
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from imblearn.over_sampling import SMOTE
import joblib

# Load dataset
df = pd.read_csv("/content/tweet_emotions.csv")

# Drop useless column and rename
df = df.drop("tweet_id", axis=1)
df = df.rename(columns={"content": "text", "sentiment": "emotion"})

# Clean text
def clean(text):
    text = nfx.remove_userhandles(text)
    text = nfx.remove_stopwords(text)
    text = nfx.remove_special_characters(text)
    return text.lower().strip()

df["clean_text"] = df["text"].apply(clean)

# OPTIONAL: Filter for most common emotions (top 5)
top_emotions = df["emotion"].value_counts().nlargest(5).index.tolist()
df = df[df["emotion"].isin(top_emotions)]

# Vectorize
vectorizer = TfidfVectorizer(max_features=5000, stop_words="english", ngram_range=(1, 2))
X = vectorizer.fit_transform(df["clean_text"])
y = df["emotion"]

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

# Balance with SMOTE
sm = SMOTE(random_state=42)
X_train_bal, y_train_bal = sm.fit_resample(X_train, y_train)

# Train model
model = LogisticRegression(max_iter=1000)
model.fit(X_train_bal, y_train_bal)

# Evaluate
y_pred = model.predict(X_test)
print("✅ Accuracy:", accuracy_score(y_test, y_pred)*100)
print("\n📄 Classification Report:\n", classification_report(y_test, y_pred))

# Save model
joblib.dump(model, "emotion_model.pkl")
joblib.dump(vectorizer, "tfidf_vectorizer.pkl")

# Predict function
def predict_emotion(text):
    text = clean(text)
    vec = vectorizer.transform([text])
    return model.predict(vec)[0]

# Sample prediction(demo)
sample = "I can't stop laughing, this is hilarious!"
print(f"Predicted Emotion: {predict_emotion(sample)}")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.7/114.7 kB 4.4 MB/s eta 0:00:00
✅ Accuracy: 40.58757783809676

📄 Classification Report:
               precision    recall  f1-score   support

   happiness       0.38      0.42      0.40      1042
        love       0.40      0.48      0.44       768
     neutral       0.43      0.41      0.42      1728
     sadness       0.34      0.36      0.35      1033
       worry       0.45      0.39      0.41      1692

    accuracy                           0.41      6263
   macro avg       0.40      0.41      0.40      6263
weighted avg       0.41      0.41      0.41      6263

Predicted Emotion: happiness
